### **LAB6**
 **Exercise2**
### **AIM: Apply Logistic Regression on MNIST dataset using Tensorflow**

Nirali Shah | CE122

In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

MNIST data set. MNIST data is a collection of hand-written digits that contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 255.

Next for each image we will:

1) converted it to float32

2) normalized to [0, 1]

3) flattened to a 1-D array of 784 features (28*28).

### Step 2: Loading and Preparing the MNIST Data Set

In [11]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)


# Flatten images to 1-D vector of 784 features (28*28).
num_features=784
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])


# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

### Step 3: Setting Up Hyperparameters and Data Set Parameters

Initialize the model parameters.

num_classes denotes the number of outputs, which is 10, as we have digits from 0 to 9 in the data set.

num_features defines the number of input parameters, and we store 784 since each image contains 784 pixels.

In [12]:
# MNIST dataset parameters.

num_classes = 10 # 0 to 9 digits

num_features = 784 # 28*28

# Training parameters.

learning_rate = 0.01

training_steps = 1000

batch_size = 256

display_step = 50

### Step 4: Shuffling and Batching the Data

We need to shuffle and batch the data before we start the actual training to avoid the model from getting biased by the data. This will allow our data to be more random and helps our model to gain higher accuracies with the test data.

With the help of tf.data.Dataset.from_tensor_slices, we can get the slices of an array in the form of objects.

The function shuffle(5000) randomizes the order of the data set’s examples.

Here, 5000 denotes the variable shuffle_buffer, which tells the model to pick a sample randomly from 1 to 5000 samples.

After that, there are only 4999 samples left in the buffer, so the sample 5001 gets added to the buffer.

In [13]:
# Use tf.data API to shuffle and batch data.
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

### Step 5: Initializing Weights and Biases

In [14]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.

W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

# Bias of shape [10], the total number of classes.

b = tf.Variable(tf.zeros([num_classes]), name="bias")

### Step 6: Defining Logistic Regression and Cost Function

We define the logistic_regression function below, which converts the inputs into a probability distribution proportional to the exponents of the inputs using the softmax function. The softmax function, which is implemented using the function tf.nn.softmax, also makes sure that the sum of all the inputs equals one.

In [15]:
# Logistic regression (Wx + b).

def logistic_regression(x):

    # Apply softmax to normalize the logits to a probability distribution.
   return tf.nn.softmax(tf.matmul(x, W) + b)
    

# Cross-Entropy loss function.

def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.
  y_true = tf.one_hot(y_true, depth=num_classes)
    

    # Clip prediction values to avoid log(0) error.
  y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    

    # Compute cross-entropy.
  return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

### Step 7: Defining Optimizers and Accuracy Metrics

When we compute the output, it gives us the probability of the given data to fit a particular class of output.

We consider the correct prediction as to the class having the highest probability.

We compute this using the function tf.argmax.

We also define the stochastic gradient descent as the optimizer from several optimizers present in TensorFlow. We do this using the function tf.optimizers.SGD.

This function takes in the learning rate as its input, which defines how fast the model should reach its minimum loss or gain the highest accuracy.

In [16]:
# Accuracy metric.

def accuracy(y_pred, y_true):

  # Predicted class is the index of the highest score in prediction vector (i.e. argmax).

  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))

  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Step 8: Optimization Process and Updating Weights and Biases

Now we define the run_optimization() method where we update the weights of our model. We calculate the predictions using the logistic_regression(x) method by taking the inputs and find out the loss generated by comparing the predicted value and the original value present in the data set. Next, we compute the gradients using and update the weights of the model with our stochastic gradient descent optimizer.

In [19]:
# Optimization process. 

def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.
   with tf.GradientTape() as g:
    pred = logistic_regression(x)
    loss = cross_entropy(pred, y)

    # Compute gradients.
    gradients = g.gradient(loss, [W, b])
    optimizer = tf.optimizers.SGD(learning_rate)
    


    # Stochastic gradient descent optimizer.

    
    
    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [W, b]))

### Step 9: The Training Loop

In [24]:
# Run training for the given number of steps.

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)

    

    if step % display_step == 0:

        pred = logistic_regression(batch_x)
       
        loss = cross_entropy(pred, batch_y)
        
        acc = accuracy(pred, batch_y)
        
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))
        

step: 50, loss: 51.977287, accuracy: 0.929688
step: 100, loss: 118.306992, accuracy: 0.843750
step: 150, loss: 59.824070, accuracy: 0.937500
step: 200, loss: 367.556824, accuracy: 0.832031
step: 250, loss: 57.288147, accuracy: 0.929688
step: 300, loss: 48.115673, accuracy: 0.937500
step: 350, loss: 35.831993, accuracy: 0.957031
step: 400, loss: 55.660057, accuracy: 0.941406
step: 450, loss: 105.890770, accuracy: 0.898438
step: 500, loss: 93.503624, accuracy: 0.921875
step: 550, loss: 82.266640, accuracy: 0.917969
step: 600, loss: 73.787247, accuracy: 0.929688
step: 650, loss: 76.126251, accuracy: 0.941406
step: 700, loss: 66.569771, accuracy: 0.925781
step: 750, loss: 25.652819, accuracy: 0.980469
step: 800, loss: 165.956543, accuracy: 0.851562
step: 850, loss: 96.615013, accuracy: 0.914062
step: 900, loss: 76.892517, accuracy: 0.921875
step: 950, loss: 51.975914, accuracy: 0.933594
step: 1000, loss: 68.386627, accuracy: 0.921875


### Step 10: Testing Model Accuracy Using the Test Data

In [25]:
# Test model on validation set.
pred = logistic_regression(x_test)

print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.917500
